In [1]:
from keras.models import load_model
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.utils import CustomObjectScope
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import h5py
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

/home/lavish/Documents/tensorflow3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
for layer in FRmodel.layers:
    layer.trainable=False

In [3]:
FRmodel.get_layer('dense_layer').trainable=True
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [4]:
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss

In [5]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))
f=h5py.File('datasets/train_face.h5','r')
print("Keys: %s" % f.keys())
train_X=list(f.keys())[0]
data = list(f[train_X])
print(data[0])

loss = 528.14294
Keys: KeysView(<HDF5 file "train_face.h5" (mode r)>)
b'kian'


In [6]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [8]:
database = {}

for folder in os.listdir('lfw/'):
    ln=len([name for name in os.listdir('lfw'+'/'+folder)])
    if ln>1:
        for file in os.listdir('lfw/'+folder):
            file_ad = 'lfw/'+folder+'/'+file
            database[file]=img_to_encoding2(file_ad,FRmodel)

/home/lavish/Documents/tensorflow3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [ ]:
# verify

def verify(image_path, identity, database, model):   
    
    encoding = img_to_encoding2(image_path,model)
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding-database[identity])
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist<0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True 
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    ### END CODE HERE ###
        
    return dist, door_open
print(os.listdir('lfw'))

In [ ]:
verify("lfw/Bryan_Chui/Bryan_Chui_0001.jpg", "Bryan_Chui", database, FRmodel)

In [ ]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

In [ ]:
# GRADED FUNCTION: who_is_it

def who_is_it(image_path,person,database, model):   

    encoding = img_to_encoding2(image_path,model)
    
    min_dist = 100
    
    for (name, db_enc) in database.items():
        
    
        dist = np.linalg.norm(encoding-db_enc)

    
        if dist<min_dist:
            min_dist = dist
            c=0
            for letter in name:
                    if(letter.isdigit()):
                        name=name[0:c-1]
                    c+=1
            identity = name

    
    if min_dist < 0.7 and identity==person:
        output=1
    else:
        output=0
        
    return min_dist, output

In [9]:
# FUNCTION: who_is_it

def who_is_it(image_path,person,database, model):   

    encoding = img_to_encoding2(image_path,model)
    
    min_dist = 100
    
    for (name, db_enc) in database.items():
        
        dist = np.linalg.norm(encoding-db_enc)

        if dist<min_dist:
            min_dist = dist
            c=0
            for letter in name:
                    if(letter.isdigit()):
                        name=name[0:c-1]
                    c+=1
            identity = name
    
    if min_dist < 0.7 and identity==person:
        output=1
    else:
        output=0
        
    return output,identity


In [10]:
total=0
acc = 0
for folder in os.listdir('lfw/'):
    ln=len([name for name in os.listdir('lfw'+'/'+folder)])
    if ln>1:
        for file in os.listdir('lfw/'+folder):
            file_ad = 'lfw/'+folder+'/'+file
            out,identity = who_is_it(file_ad,folder,database,FRmodel)
            total+=1
            acc+=out
print(str(acc*100/total)+'% is total acccuracy')

/home/lavish/Documents/tensorflow3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


100.0% is total acccuracy


In [11]:
print(total)

9164
